Отлично 👍
Тогда давай начнём с **Модуля 1 — Введение в мониторинг**, и я разберу его так, как будто мы сидим в аудитории, и я объясняю студентам, с примерами и небольшими практическими задачами.

---

# 📘 Модуль 1 — Введение в мониторинг

## 🎯 Цель модуля

* Понять, зачем в принципе нужен мониторинг.
* Разобраться в основных типах данных: логи, метрики, трассировки.
* Понять разницу между подходами Pull и Push.
* Осознать, какое место Prometheus и Grafana занимают в DevOps-экосистеме.

---

## 1. Что такое мониторинг и зачем он нужен

Представь, что у тебя есть **приложение**, которое обслуживает клиентов. Всё работает… пока не перестанет.

**Мониторинг — это "система раннего предупреждения"**, которая позволяет:

* **Замечать проблемы до того, как они ударят по пользователям.**
* **Собирать данные для анализа** — например, почему сервис упал.
* **Понимать нагрузку** — когда пора масштабировать.
* **Ставить автоматические алерты** — например, если CPU 95% в течение 5 минут.

💡 **Аналогия**: Мониторинг в IT — это приборная панель автомобиля.
Если у тебя нет спидометра, датчика топлива и лампочек ошибок двигателя, ты можешь ехать… но в один момент машина просто встанет.

---

## 2. Логи, метрики, трассировки — в чём разница?

В мониторинге есть три "кита":

| Вид             | Что это                                | Пример                                              |
| --------------- | -------------------------------------- | --------------------------------------------------- |
| **Логи**        | Текстовые записи событий               | `2025-08-11 10:12:34 ERROR DB connection failed`    |
| **Метрики**     | Числовые данные, измеряемые во времени | CPU: 45%, Memory: 2.3 GB                            |
| **Трассировки** | Путь запроса через систему             | Запрос пользователя прошёл 3 микросервиса за 124 мс |

**Prometheus** → собирает **метрики**.
**Grafana** → визуализирует их, но может и логи показывать, если подключить Loki.

---

## 3. Подходы к сбору данных: Push vs Pull

**Push-модель**

* Приложение **отправляет данные** в систему мониторинга.
* Пример: клиент шлёт POST-запрос в InfluxDB.
* Минус: сложнее контролировать источники данных.

**Pull-модель**

* Система мониторинга **сама ходит** за данными по определённым адресам (targets).
* Пример: Prometheus опрашивает Node Exporter каждые 15 секунд.
* Плюс: легко централизованно управлять конфигурацией.

💡 **Prometheus использует Pull** — это даёт контроль, версионирование и лёгкое масштабирование.

---

## 4. Место Prometheus и Grafana в DevOps-экосистеме

Если нарисовать схему DevOps-инструментов, то:

* **Prometheus** → сбор и хранение метрик
* **Alertmanager** → уведомления и алерты
* **Grafana** → красивая визуализация и дашборды
* **Loki** → логи
* **Jaeger** → трассировки

📌 Пример потока:

1. Приложение отдаёт метрики → Node Exporter → Prometheus
2. Prometheus хранит данные
3. Alertmanager отправляет сообщение в Telegram при проблеме
4. Grafana показывает графики

---

## 5. Мини-практика

**Задача:** Определи, что здесь — метрика, что лог, а что трассировка:

1. `"2025-08-11 14:21:02 GET /api/orders 500 Internal Server Error"`
2. `cpu_usage_percent{instance="server1"} = 87.3`
3. "Запрос от пользователя 123 прошёл сервис auth → orders → payment за 348 мс"

**Ответ:**

1. Лог
2. Метрика
3. Трассировка

---

## 6. Домашнее задание

1. Приведи 3 примера метрик, которые полезно собирать для:

   * Веб-сайта
   * Интернет-магазина
   * Микросервисной системы
2. Нарисуй простую схему (от руки или в draw\.io), как ты представляешь сбор метрик с сервера через Prometheus и отображение в Grafana.

---

📍 На следующем занятии (**Модуль 2**) мы разберём архитектуру Prometheus и то, как все его компоненты взаимодействуют.
Я дам подробную схему, покажу, куда подключаются экспортеры, и объясню, как идёт поток данных.

---

Если хочешь, я могу сразу перейти к **Модулю 2 — Архитектура Prometheus** и сделать его с детальной схемой и жизненным примером.
Хочешь продолжим?


Отлично, тогда продолжаем 🚀

---

# 📘 Модуль 2 — Архитектура Prometheus

## 🎯 Цель модуля

* Разобраться, из каких компонентов состоит Prometheus.
* Понять, как идёт поток данных от приложения до графика в Grafana.
* Увидеть, где в этой схеме находятся экспортеры и Alertmanager.
* Подготовиться к установке Prometheus.

---

## 1. Общая идея работы Prometheus

Prometheus — это **система мониторинга и хранения временных рядов (time series)**.
Его задача: **периодически опрашивать источники данных** (targets), забирать оттуда метрики в формате **метка → значение → время** и хранить их для анализа и визуализации.

💡 Пример временного ряда:

```
http_requests_total{method="GET", status="200"} 15423  1691745600000
http_requests_total{method="GET", status="200"} 15425  1691745615000
```

Тут:

* **Имя метрики** — `http_requests_total`
* **Лейблы** — `method="GET"`, `status="200"`
* **Значение** — 15423, 15425
* **Время** — в миллисекундах

---

## 2. Компоненты архитектуры Prometheus

### **1. Prometheus Server**

* Сердце системы.
* Выполняет **pull-запросы** к целям (`targets`), чтобы собрать метрики.
* Хранит метрики в своей базе (TSDB — time series database).
* Обрабатывает запросы PromQL.
* Передаёт алерты в Alertmanager.

---

### **2. Exporters**

Prometheus не лезет в твой код напрямую — он ходит к экспортеру.

**Экспортер** — это сервис, который:

* Снимает метрики с системы/приложения
* Отдаёт их в формате, который понимает Prometheus (обычно `text/plain` HTTP endpoint `/metrics`)

Примеры:

* **Node Exporter** — метрики ОС (CPU, память, диски, сеть)
* **Blackbox Exporter** — проверка доступности сайтов, API
* **cAdvisor** — метрики Docker-контейнеров
* **Custom Exporter** — твой код с `prometheus_client` библиотекой

---

### **3. Alertmanager**

* Получает от Prometheus сигналы о нарушении правил (алерты).
* Группирует, подавляет, маршрутизирует алерты.
* Отправляет уведомления в Slack, Telegram, Email и т.д.
* Работает отдельно, но связан с Prometheus.

---

### **4. Pushgateway**

* Принимает метрики от приложений по **push**-подходу (когда pull невозможен, например, у короткоживущих batch-скриптов).
* Хранит эти метрики, чтобы Prometheus мог их забрать.
* Используется редко в долгоживущих сервисах.

---

## 3. Поток данных (жизненный цикл метрики)

Давай представим, что у нас есть микросервис **orders-service**, и мы хотим видеть, сколько заказов обрабатывается в минуту.

**Схема процесса:**

1. **Orders-service** → отдаёт `/metrics` (через Prometheus Client)
2. **Prometheus** → каждые 15 секунд ходит по `/metrics` и забирает данные
3. Данные сохраняются в **TSDB** на диске Prometheus
4. Если правило алерта сработало — Prometheus шлёт событие в **Alertmanager**
5. Alertmanager отправляет сообщение в **Telegram**
6. **Grafana** ходит в Prometheus за данными и строит график

---

📌 **Визуальная схема (упрощённая)**

```
[orders-service]   [node-exporter]       [cAdvisor]
     |                   |                  |
     +---------+---------+--------+---------+
               |                  |
           [Prometheus Server] <--+  (Pull)
               |   ^   \
               |   |    \
               |   |     +--> [Alertmanager] -> (Telegram, Slack)
               v
           [Grafana UI]
```

---

## 4. Как Prometheus понимает, откуда забирать метрики

Это задаётся в конфиге `prometheus.yml`:

```yaml
scrape_configs:
  - job_name: 'node'
    static_configs:
      - targets: ['192.168.1.10:9100', '192.168.1.11:9100']

  - job_name: 'orders-service'
    static_configs:
      - targets: ['orders-service:8000']
```

Здесь:

* **job\_name** — имя задачи (логическое)
* **targets** — список адресов с `/metrics`

---

## 5. Мини-практика

**Задача:**
Представь, что у тебя есть три сервера:

* `api-server` с приложением на порту 8080 (отдаёт `/metrics`)
* `node-exporter` на каждом сервере (порт 9100)
* `db-server` с PostgreSQL и экспортёром `postgres_exporter` на порту 9187

Напиши, как будут выглядеть `targets` в `prometheus.yml`.

---

**Возможный ответ:**

```yaml
scrape_configs:
  - job_name: 'node'
    static_configs:
      - targets: ['api-server:9100', 'db-server:9100']

  - job_name: 'api'
    static_configs:
      - targets: ['api-server:8080']

  - job_name: 'postgres'
    static_configs:
      - targets: ['db-server:9187']
```

---

## 6. Домашнее задание

1. Нарисуй схему архитектуры Prometheus для своей будущей системы.
2. Определи, какие экспортеры тебе понадобятся.
3. Подумай, где тебе придётся использовать Pushgateway, а где Pull будет достаточно.

---

📍 В **Модуле 3** мы уже перейдём к практике: установим Prometheus, напишем свой первый `prometheus.yml`, и я покажу, как прямо в браузере посмотреть метрики.

---

Если хочешь, я могу **сразу сделать Модуль 3 с установкой Prometheus и первым запуском в Docker**.
Продолжим?


Отлично, тогда идём дальше 🚀

---

# 📘 Модуль 3 — Установка и первый запуск Prometheus

## 🎯 Цель модуля

* Установить Prometheus локально (в Docker или напрямую).
* Написать первый `prometheus.yml`.
* Запустить сервер и открыть веб-интерфейс.
* Убедиться, что он уже собирает метрики.

---

## 1. Способы установки Prometheus

Prometheus можно развернуть тремя основными способами:

1. **Через Docker** — быстро, удобно, подходит для обучения и тестов.
2. **Через бинарники** — скачиваем с [prometheus.io](https://prometheus.io/download/) и запускаем вручную.
3. **Через Kubernetes Helm Chart** — для продакшена в k8s.

💡 Для курса будем использовать **Docker**, так как это быстрее всего.

---

## 2. Создаём конфигурацию `prometheus.yml`

Prometheus сам по себе ничего не мониторит — нужно сказать, куда ему ходить.
В конфиге описываются `jobs` и `targets`.

📄 **Файл** `prometheus.yml`:

```yaml
global:
  scrape_interval: 15s   # как часто опрашивать цели
  evaluation_interval: 15s # как часто пересчитывать правила

scrape_configs:
  - job_name: 'prometheus'
    static_configs:
      - targets: ['localhost:9090']
```

Здесь мы указали, что будем мониторить сам Prometheus (он тоже отдаёт свои метрики на `/metrics` по 9090 порту).

---

## 3. Запуск Prometheus в Docker

1. **Создаём папку проекта**

```bash
mkdir prometheus-demo
cd prometheus-demo
```

2. **Кладём туда `prometheus.yml`**

```
prometheus-demo/
 ├── prometheus.yml
```

3. **Запускаем контейнер**

```bash
docker run -d \
  --name prometheus \
  -p 9090:9090 \
  -v $(pwd)/prometheus.yml:/etc/prometheus/prometheus.yml \
  prom/prometheus
```

* `-p 9090:9090` — пробрасываем порт.
* `-v ...` — монтируем наш конфиг в контейнер.

---

## 4. Проверяем работу

1. Открываем в браузере:

```
http://localhost:9090
```

2. Видим веб-интерфейс Prometheus.
3. Вводим в строку запроса:

```
up
```

Это базовая метрика — показывает, какие цели доступны (`1` = доступна, `0` = нет).

---

## 5. Мини-практика

**Задача:** Добавь в `prometheus.yml` ещё один job для мониторинга **Node Exporter** на `localhost:9100` (мы пока его не ставили, но подготовим конфиг).

**Решение:**

```yaml
scrape_configs:
  - job_name: 'prometheus'
    static_configs:
      - targets: ['localhost:9090']

  - job_name: 'node'
    static_configs:
      - targets: ['localhost:9100']
```

---

## 6. Домашнее задание

1. Запусти Prometheus у себя в Docker.
2. Открой веб-интерфейс и выполни запрос:

   ```
   prometheus_http_requests_total
   ```

   Посмотри, какие лейблы там есть.
3. Подумай, какие ещё сервисы ты бы хотел добавить для мониторинга в будущем.

---

📍 В **Модуле 4** мы займёмся **PromQL** — языком запросов Prometheus.
Я покажу, как доставать данные, фильтровать их, строить агрегаты и готовить к визуализации в Grafana.

---

Если хочешь, могу **сразу сделать Модуль 4 с разбором PromQL на примерах и мини-заданиями**.
Продолжим?


Отлично, тогда погружаемся в PromQL 🖥️📊

---

# 📘 Модуль 4 — Основы PromQL

## 🎯 Цель модуля

* Понять, что такое временные ряды (time series) и как их запрашивать.
* Научиться фильтровать метрики по лейблам.
* Разобраться в типах данных Prometheus.
* Освоить базовые функции (`rate`, `increase`, `avg_over_time`).
* Уметь агрегировать данные (`sum`, `avg`, `max`, `min`, `count`).

---

## 1. Что такое PromQL

PromQL (**Prometheus Query Language**) — язык запросов к хранилищу временных рядов Prometheus.
Он позволяет:

* Выбирать метрики по имени
* Фильтровать по лейблам
* Строить агрегаты
* Считать производные величины (например, скорость роста счётчика)
* Работать с интервалами времени

💡 **Важный момент**: в Prometheus **все данные привязаны к моменту времени**.

---

## 2. Типы данных PromQL

| Тип                | Что это                            | Пример   |
| ------------------ | ---------------------------------- | -------- |
| **Instant Vector** | Набор метрик в один момент времени | `up`     |
| **Range Vector**   | Набор метрик за интервал времени   | `up[5m]` |
| **Scalar**         | Одно число без лейблов             | `5`      |
| **String**         | Строка (почти не используется)     | `"OK"`   |

---

## 3. Первый запрос: `up`

Метрика `up` встроена в Prometheus и показывает, доступны ли цели:

* `1` — цель отвечает
* `0` — цель недоступна

Пример:

```
up
```

Результат:

```
up{instance="localhost:9090", job="prometheus"}  1
```

---

## 4. Фильтрация по лейблам

**Синтаксис**:

```promql
метрика{лейбл="значение"}
```

Примеры:

```promql
up{job="prometheus"}
up{instance="localhost:9090"}
```

**Операторы**:

* `=` — равно
* `!=` — не равно
* `=~` — соответствует regex
* `!~` — не соответствует regex

Пример с regex:

```promql
up{job=~"prometheus|node"}
```

---

## 5. Агрегация

Prometheus умеет агрегировать значения по меткам.

Пример:

```promql
sum(up)        # Сумма значений up по всем целям
avg(up)        # Среднее
count(up)      # Кол-во серий
max(up)        # Максимум
min(up)        # Минимум
```

Можно указывать `by` и `without`:

```promql
sum(up) by (job)
avg(up) without (instance)
```

---

## 6. Функции PromQL

### **`rate()`** — скорость изменения счётчика в секунду

```promql
rate(http_requests_total[5m])
```

Покажет среднюю скорость роста за последние 5 минут.

---

### **`increase()`** — сколько прибавилось за интервал

```promql
increase(http_requests_total[5m])
```

Если за 5 минут пришло 200 запросов, вернёт `200`.

---

### **`avg_over_time()`** — среднее значение за интервал

```promql
avg_over_time(node_cpu_seconds_total[1h])
```

---

💡 **Разница между `rate` и `increase`**:

* `rate` → в единицах **в секунду**
* `increase` → в **штуках за интервал**

---

## 7. Мини-практика

**Задача:**
Напиши PromQL-запросы для:

1. Показать все цели, которые недоступны (`up` = 0).
2. Посчитать среднюю загрузку CPU за последние 5 минут по всем серверам.
3. Найти количество HTTP-запросов за последние 10 минут.

**Решение:**

```promql
# 1
up == 0

# 2 (пример для node_exporter)
avg(rate(node_cpu_seconds_total{mode="user"}[5m])) by (instance)

# 3
increase(http_requests_total[10m])
```

---

## 8. Домашнее задание

1. Найди в своём Prometheus 3 метрики, у которых есть лейблы, и потренируйся фильтровать их.
2. Сделай запрос, который покажет суммарную скорость роста любой счётчик-метрики за последние 1 минуту.
3. Попробуй сделать `sum` по этой метрике с `by (job)`.

---

📍 В **Модуле 5** мы перейдём к **экспортерам** — я расскажу, какие бывают, зачем они нужны, и мы подключим первый **Node Exporter** для метрик сервера.

---

Хочешь, чтобы я сделал **Модуль 5 с установкой Node Exporter и его подключением к Prometheus** прямо сейчас?


Отлично, тогда идём дальше 🚀

---

# 📘 Модуль 5 — Экспортеры и подключение Node Exporter

## 🎯 Цель модуля

* Понять, что такое экспортеры и зачем они нужны.
* Узнать, какие бывают популярные экспортеры.
* Установить и запустить Node Exporter.
* Подключить его к Prometheus и посмотреть метрики.

---

## 1. Что такое экспортер

**Экспортер** — это программа, которая собирает метрики из какой-то системы и отдаёт их в формате, который понимает Prometheus.
Prometheus сам в систему не лезет, а опрашивает экспортер по HTTP.

💡 **Аналогия:** Экспортер — это переводчик, который превращает данные из “языка” системы в язык Prometheus.

---

## 2. Популярные экспортеры

| Экспортер             | Что мониторит                               | Порт по умолчанию |
| --------------------- | ------------------------------------------- | ----------------- |
| **Node Exporter**     | Операционная система (CPU, RAM, диск, сеть) | 9100              |
| **Blackbox Exporter** | Доступность сервисов (ping, HTTP, TCP)      | 9115              |
| **cAdvisor**          | Docker-контейнеры                           | 8080              |
| **Postgres Exporter** | PostgreSQL                                  | 9187              |
| **MySQL Exporter**    | MySQL                                       | 9104              |
| **Redis Exporter**    | Redis                                       | 9121              |

---

## 3. Установка Node Exporter (в Docker)

Node Exporter — это самый базовый и популярный экспортер для серверов.

📄 **Запуск в Docker**:

```bash
docker run -d \
  --name node-exporter \
  -p 9100:9100 \
  --restart unless-stopped \
  prom/node-exporter
```

Теперь метрики доступны по адресу:

```
http://localhost:9100/metrics
```

Открой в браузере — увидишь текстовый список метрик.

---

## 4. Подключение Node Exporter к Prometheus

1. Открываем `prometheus.yml`.
2. Добавляем новый `job`:

```yaml
scrape_configs:
  - job_name: 'prometheus'
    static_configs:
      - targets: ['localhost:9090']

  - job_name: 'node'
    static_configs:
      - targets: ['localhost:9100']
```

3. Перезапускаем Prometheus (если в Docker):

```bash
docker restart prometheus
```

4. Заходим в веб-интерфейс Prometheus → **Status → Targets**.
   Там должны быть:

* `prometheus` — `UP`
* `node` — `UP`

---

## 5. Первые запросы к Node Exporter

**Примеры метрик:**

```promql
node_cpu_seconds_total           # время работы CPU по режимам
node_memory_MemAvailable_bytes   # доступная память
node_network_receive_bytes_total # принятые байты по сети
node_disk_read_bytes_total       # прочитано с диска
```

---

## 6. Мини-практика

**Задача:**

1. Найди метрику, которая показывает использование CPU в процентах.
2. Выведи среднее использование CPU за последние 5 минут по каждому серверу.

**Решение:**

```promql
100 - (avg by (instance) (rate(node_cpu_seconds_total{mode="idle"}[5m])) * 100)
```

---

## 7. Домашнее задание

1. Запусти Node Exporter у себя.
2. Добавь его в `prometheus.yml` и убедись, что он в состоянии `UP`.
3. Найди метрики для:

   * свободной оперативной памяти,
   * использования диска,
   * скорости передачи данных по сети.
4. Построй для них графики прямо в интерфейсе Prometheus.

---

📍 В **Модуле 6** мы займёмся **Alertmanager** — настроим систему оповещений, чтобы получать уведомления в Telegram при проблемах (например, когда CPU > 90%).

---

Хочешь, чтобы я сразу сделал **Модуль 6 с установкой Alertmanager и настройкой алертов в Telegram**?


Отлично, идём к самому «живому» этапу — автоматическим оповещениям 📢💬

---

# 📘 Модуль 6 — Alertmanager и оповещения в Telegram

## 🎯 Цель модуля

* Понять, как работает Alertmanager.
* Установить и запустить его.
* Настроить правила оповещений в Prometheus.
* Отправлять уведомления в Telegram при срабатывании алертов.

---

## 1. Как работает Alertmanager

Prometheus сам **не отправляет** уведомления — он только проверяет условия.
Когда условие срабатывает:

1. Prometheus отправляет событие в Alertmanager.
2. Alertmanager решает, **куда** отправить уведомление (Telegram, email, Slack, Webhook).
3. Alertmanager умеет группировать события и подавлять лишние (например, чтобы не спамить).

💡 **Формула:**
Prometheus → (триггер) → Alertmanager → (доставка) → Telegram

---

## 2. Установка Alertmanager (Docker)

```bash
docker run -d \
  --name alertmanager \
  -p 9093:9093 \
  --restart unless-stopped \
  -v $(pwd)/alertmanager.yml:/etc/alertmanager/alertmanager.yml \
  prom/alertmanager
```

---

## 3. Настройка подключения Prometheus ↔ Alertmanager

В `prometheus.yml` добавляем:

```yaml
alerting:
  alertmanagers:
    - static_configs:
        - targets:
            - 'localhost:9093'
```

---

## 4. Создание правил алертов в Prometheus

Создадим файл `alerts.yml`:

```yaml
groups:
  - name: example-alerts
    rules:
      - alert: HighCPUUsage
        expr: 100 - (avg by (instance) (rate(node_cpu_seconds_total{mode="idle"}[5m])) * 100) > 90
        for: 1m
        labels:
          severity: critical
        annotations:
          summary: "Высокая загрузка CPU на {{ $labels.instance }}"
          description: "CPU > 90% в течение 1 минуты."
```

В `prometheus.yml` подключаем этот файл:

```yaml
rule_files:
  - "alerts.yml"
```

---

## 5. Подготовка Telegram-бота

1. В Telegram напиши боту [@BotFather](https://t.me/BotFather).
2. Создай нового бота: `/newbot`.
3. Получи **API Token**.
4. Найди свой **chat\_id** через бота [@userinfobot](https://t.me/userinfobot).

---

## 6. Конфигурация Alertmanager для Telegram

Файл `alertmanager.yml`:

```yaml
route:
  receiver: 'telegram'

receivers:
  - name: 'telegram'
    telegram_configs:
      - bot_token: 'ТОКЕН_БОТА'
        chat_id: CHAT_ID
        message: |
          🔴 АЛЕРТ: {{ .CommonAnnotations.summary }}
          {{ .CommonAnnotations.description }}
```

---

## 7. Запуск и тест

1. Перезапускаем Alertmanager:

```bash
docker restart alertmanager
```

2. Перезапускаем Prometheus:

```bash
docker restart prometheus
```

3. Спровоцируй алерт:

   * Либо нагрузи CPU:

     ```bash
     yes > /dev/null
     ```
   * Либо временно уменьши порог в `expr` до `> 1` и подожди минуту.

4. Должно прийти уведомление в Telegram.

---

## 8. Мини-практика

**Задачи:**

1. Сделать алерт, если свободной оперативной памяти < 500 МБ.
2. Сделать алерт, если диск заполнен более чем на 90%.
3. Сделать алерт, если `node_exporter` перестал отвечать.

---

## 9. Домашнее задание

1. Настроить Alertmanager в связке с Prometheus.
2. Подключить Telegram и проверить, что оповещения приходят.
3. Сделать хотя бы 2 собственных правила алертов.

---

📍 В **Модуле 7** мы начнём **Grafana** — подключим Prometheus, создадим первый дашборд и научимся строить красивые графики.

---

Хочешь, чтобы я сделал **Модуль 7 про подключение Grafana и первый дашборд** прямо сейчас?


Отлично, идём к визуализации 📊✨

---

# 📘 Модуль 7 — Подключение Grafana и создание первого дашборда

## 🎯 Цель модуля

* Установить Grafana.
* Подключить Prometheus как источник данных.
* Создать первый дашборд с графиками.
* Освоить базовые возможности панелей и фильтров.

---

## 1. Установка Grafana (Docker)

```bash
docker run -d \
  --name=grafana \
  -p 3000:3000 \
  --restart unless-stopped \
  grafana/grafana
```

После запуска Grafana будет доступна по адресу:

```
http://localhost:3000
```

**Логин:** `admin`
**Пароль:** `admin` (при первом входе попросит сменить).

---

## 2. Подключение Prometheus как источника данных

1. В Grafana перейди: **Settings → Data sources → Add data source**.
2. Выбери **Prometheus**.
3. В поле **URL** укажи:

```
http://host.docker.internal:9090
```

> Если Grafana и Prometheus в одной Docker-сети, то можно указать имя контейнера, например:

```
http://prometheus:9090
```

4. Нажми **Save & Test** — должно быть зелёное сообщение *Data source is working*.

---

## 3. Создание первого дашборда

1. **Dashboards → New → New dashboard**.
2. **Add visualization**.
3. Выбираем **Prometheus** как источник.
4. В поле запроса вводим, например:

```promql
100 - (avg by (instance) (rate(node_cpu_seconds_total{mode="idle"}[5m])) * 100)
```

Это загрузка CPU в %.
5\. Выбираем тип графика — **Time series**.
6\. Сохраняем панель.

---

## 4. Примеры метрик для графиков

* **CPU загрузка (%):**

```promql
100 - (avg by (instance) (rate(node_cpu_seconds_total{mode="idle"}[5m])) * 100)
```

* **Свободная память (ГБ):**

```promql
node_memory_MemAvailable_bytes / 1024 / 1024 / 1024
```

* **Использование диска (%):**

```promql
(1 - (node_filesystem_avail_bytes{fstype!~"tmpfs|overlay"} / node_filesystem_size_bytes{fstype!~"tmpfs|overlay"})) * 100
```

* **Сетевой трафик (MB/s):**

```promql
rate(node_network_receive_bytes_total[5m]) / 1024 / 1024
```

---

## 5. Настройка дашборда

* Можно добавить **переменные** (variables) для выбора сервера, интерфейса или диска.
* Можно менять цветовые схемы, легенду, алерты прямо в панелях.
* Дашборды можно экспортировать и импортировать (JSON).

---

## 6. Готовые дашборды Grafana

Grafana имеет **официальную библиотеку дашбордов**:
[https://grafana.com/grafana/dashboards/](https://grafana.com/grafana/dashboards/)

Пример: дашборд **Node Exporter Full** (ID: `1860`) — уже готовая панель с CPU, памятью, дисками и сетью.

**Как импортировать:**

1. **Dashboards → Import**.
2. Ввести ID `1860`.
3. Указать источник данных — ваш Prometheus.
4. Готово 🎉

---

## 7. Мини-практика

1. Создай дашборд с 3 графиками:

   * CPU загрузка.
   * Свободная память.
   * Заполненность диска.
2. Импортируй готовый дашборд `1860` и посмотри, как он устроен.
3. Добавь фильтр для выбора сервера (переменная `instance`).

---

## 8. Домашнее задание

1. Установить Grafana и подключить Prometheus.
2. Создать хотя бы один дашборд с 3–4 панелями.
3. Настроить переменные для выбора сервера.
4. Импортировать и изучить готовый дашборд из библиотеки.

---

📍 В **Модуле 8** мы сделаем:

* Алерты прямо в Grafana.
* Красивые панельки с порогами.
* Дашборд для бизнес-метрик (не только тех.метрик).

---

Хочешь, чтобы я подготовил **Модуль 8 с алертами и продвинутой настройкой панелей в Grafana**?


Отлично, идём к продвинутой магии в Grafana 🚀📊

---

# 📘 Модуль 8 — Алерты и продвинутая настройка панелей в Grafana

## 🎯 Цель модуля

* Создать алерты прямо в панелях Grafana.
* Настроить условия, пороги и уведомления.
* Освоить динамические переменные и фильтры.
* Сделать дашборд с бизнес-метриками.

---

## 1. Как работают алерты в Grafana

Сейчас в Grafana есть **Unified Alerting** — это общая система, которая:

* Работает поверх любого источника данных (Prometheus, Loki, InfluxDB).
* Может слать уведомления в Slack, Telegram, Email и др.
* Умеет объединять условия (например, CPU > 90% и RAM < 500 МБ).

💡 **Разница с Alertmanager**:

* Alertmanager управляет оповещениями для *Prometheus-алертов*.
* Grafana Alerting — управляет *дашборд-алертами*.

---

## 2. Включение Unified Alerting

1. В Grafana (v8+ по умолчанию включено) зайти в **Alerting → Contact points**.
2. Создать контакт:

   * Тип: **Telegram**.
   * Bot Token и Chat ID — те же, что мы использовали в Модуле 6.
3. Нажать **Test** — должно прийти сообщение в Telegram.

---

## 3. Создание алерта в панели

1. Открыть нужный график → **Edit**.
2. Перейти на вкладку **Alert** → **Create alert**.
3. Указать условия:

   * **Query**: используем, например:

     ```promql
     100 - (avg by (instance) (rate(node_cpu_seconds_total{mode="idle"}[5m])) * 100)
     ```
   * **Condition**: `IS ABOVE 90`.
   * Время срабатывания: `for 1m`.
4. В **Notifications** выбрать контакт (Telegram).
5. Сохранить панель.

Теперь, если CPU > 90% в течение минуты, придёт алерт.

---

## 4. Цветовые пороги и визуализация

В панели **Field → Thresholds**:

* Добавляем уровни (например, зелёный < 60%, жёлтый 60–90%, красный > 90%).
* Включаем **Color mode → Background** для наглядности.

---

## 5. Динамические переменные (Variables)

Пример — фильтр по серверу:

1. **Dashboard settings → Variables → Add variable**.
2. **Name**: `instance`.
3. **Type**: Query.
4. **Data source**: Prometheus.
5. **Query**:

   ```promql
   label_values(node_cpu_seconds_total, instance)
   ```
6. На панелях в PromQL используем:

   ```promql
   ...{instance="$instance"}
   ```

Теперь можно выбирать сервер из выпадающего списка.

---

## 6. Бизнес-метрики в Grafana

Grafana умеет показывать не только тех.метрики, но и данные из БД или API.

Примеры:

* Кол-во заказов за час (из PostgreSQL).
* Среднее время ответа API (из Prometheus).
* Выручка в реальном времени (из Google Sheets или JSON API).

**Идея**: объединить тех. и бизнес-метрики в одном дашборде, чтобы понимать, что перегрузка CPU связана с ростом заказов.

---

## 7. Мини-практика

**Задачи:**

1. Сделай панель с алертом в Grafana (CPU или RAM).
2. Настрой цветовые пороги и фоновое выделение.
3. Добавь переменную для выбора сервера.
4. Создай панель с бизнес-метрикой (можно тестовую).

---

## 8. Домашнее задание

1. Настроить Telegram-оповещения из Grafana.
2. Сделать хотя бы один алерт с порогами.
3. Добавить переменные в дашборд.
4. Сделать дашборд, где есть и тех., и бизнес-метрика.

---

📍 В **Модуле 9** мы:

* Развернём Prometheus + Alertmanager + Grafana в Docker Compose.
* Сделаем готовый стек мониторинга «под ключ».
* Добавим Node Exporter, cAdvisor и алерты.

---

Хочешь, чтобы я подготовил **Модуль 9 со стеком мониторинга в Docker Compose** прямо сейчас?


Отлично, собираем всё в единый стек 🚀📦

---

# 📘 Модуль 9 — Полный стек мониторинга (Prometheus + Alertmanager + Grafana) в Docker Compose

## 🎯 Цель модуля

* Развернуть полный мониторинг в Docker Compose.
* Включить Node Exporter и cAdvisor для системных и контейнерных метрик.
* Настроить алерты через Alertmanager и Telegram.
* Получить готовый рабочий набор, который можно развернуть на любом сервере.

---

## 1. Структура проекта

📂 monitoring
├── docker-compose.yml
├── prometheus.yml
├── alertmanager.yml

---

## 2. Файл `docker-compose.yml`

```yaml
version: '3.8'

services:
  prometheus:
    image: prom/prometheus
    container_name: prometheus
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
    command:
      - '--config.file=/etc/prometheus/prometheus.yml'
    ports:
      - "9090:9090"
    restart: unless-stopped

  alertmanager:
    image: prom/alertmanager
    container_name: alertmanager
    volumes:
      - ./alertmanager.yml:/etc/alertmanager/config.yml
    command:
      - '--config.file=/etc/alertmanager/config.yml'
    ports:
      - "9093:9093"
    restart: unless-stopped

  grafana:
    image: grafana/grafana
    container_name: grafana
    ports:
      - "3000:3000"
    restart: unless-stopped

  node_exporter:
    image: prom/node-exporter
    container_name: node_exporter
    ports:
      - "9100:9100"
    restart: unless-stopped

  cadvisor:
    image: gcr.io/cadvisor/cadvisor:latest
    container_name: cadvisor
    ports:
      - "8080:8080"
    volumes:
      - /:/rootfs:ro
      - /var/run:/var/run:ro
      - /sys:/sys:ro
      - /var/lib/docker/:/var/lib/docker:ro
    restart: unless-stopped
```

---

## 3. Файл `prometheus.yml`

```yaml
global:
  scrape_interval: 15s

alerting:
  alertmanagers:
    - static_configs:
        - targets: ['alertmanager:9093']

rule_files:
  - "alert_rules.yml"

scrape_configs:
  - job_name: 'prometheus'
    static_configs:
      - targets: ['prometheus:9090']

  - job_name: 'node_exporter'
    static_configs:
      - targets: ['node_exporter:9100']

  - job_name: 'cadvisor'
    static_configs:
      - targets: ['cadvisor:8080']
```

---

## 4. Файл `alertmanager.yml` (с Telegram)

```yaml
route:
  receiver: 'telegram'
  group_wait: 10s
  group_interval: 30s
  repeat_interval: 5m

receivers:
  - name: 'telegram'
    telegram_configs:
      - bot_token: 'ВАШ_BOT_TOKEN'
        chat_id: ВАШ_CHAT_ID
        message: |
          [ALERT] {{ .CommonAnnotations.summary }}
          Severity: {{ .CommonLabels.severity }}
          Instance: {{ .CommonLabels.instance }}
```

---

## 5. Файл `alert_rules.yml`

```yaml
groups:
  - name: system_alerts
    rules:
      - alert: HighCPUUsage
        expr: 100 - (avg by (instance) (rate(node_cpu_seconds_total{mode="idle"}[5m])) * 100) > 90
        for: 1m
        labels:
          severity: critical
        annotations:
          summary: "CPU usage > 90% on {{ $labels.instance }}"
```

---

## 6. Запуск стека

```bash
docker compose up -d
```

---

## 7. Результат

* **Prometheus** — `http://localhost:9090`
* **Grafana** — `http://localhost:3000`
* **Alertmanager** — `http://localhost:9093`
* **Node Exporter** — метрики хоста
* **cAdvisor** — метрики контейнеров

---

## 8. Мини-практика

1. Запусти стек на своём сервере или локально.
2. В Grafana подключи Prometheus как источник.
3. Импортируй дашборд **Node Exporter Full (ID 1860)**.
4. Сымитируй высокую загрузку CPU (`yes > /dev/null`) и проверь, придёт ли алерт в Telegram.

---

📍 В **Модуле 10** я могу дать:

* Оптимизацию и безопасность стека.
* Мониторинг нескольких серверов.
* Автоматическое резервное копирование конфигов и данных Grafana.

---

Хочешь, чтобы я подготовил **Модуль 10 по оптимизации и защите стека мониторинга**?


Отлично, доводим наш стек до уровня «боевого» 💪🛡️

---

# 📘 Модуль 10 — Оптимизация и защита стека мониторинга

## 🎯 Цель модуля

* Сделать стек Prometheus + Alertmanager + Grafana более безопасным.
* Оптимизировать потребление ресурсов.
* Добавить резервное копирование.
* Подготовить мониторинг для работы в продакшене.

---

## 1. Безопасность Grafana

🔹 **Меняем стандартный логин/пароль**
По умолчанию:

* Логин: `admin`
* Пароль: `admin`

Меняем в `docker-compose.yml`:

```yaml
grafana:
  image: grafana/grafana
  environment:
    - GF_SECURITY_ADMIN_USER=superadmin
    - GF_SECURITY_ADMIN_PASSWORD=сложный_пароль
```

---

🔹 **Включаем HTTPS**

* Можно поставить **Traefik** или **Nginx** в качестве reverse-proxy.
* Генерируем SSL через Let's Encrypt.

Пример Nginx-конфига с HTTPS:

```nginx
server {
    listen 443 ssl;
    server_name monitoring.example.com;

    ssl_certificate /etc/letsencrypt/live/monitoring.example.com/fullchain.pem;
    ssl_certificate_key /etc/letsencrypt/live/monitoring.example.com/privkey.pem;

    location / {
        proxy_pass http://grafana:3000;
    }
}
```

---

## 2. Оптимизация Prometheus

🔹 **Ограничиваем хранение метрик**

```yaml
command:
  - '--storage.tsdb.retention.time=15d'
  - '--storage.tsdb.path=/prometheus'
  - '--web.enable-lifecycle'
```

💡 Меньший срок хранения = меньше нагрузка на диск.

---

🔹 **Снижаем частоту опроса**
Если у нас не критичны данные каждые 15 секунд, можно:

```yaml
global:
  scrape_interval: 30s
```

Это уменьшит нагрузку на Prometheus и сетевой трафик.

---

## 3. Резервное копирование Grafana

Grafana хранит дашборды и настройки в `/var/lib/grafana`.
Пример бэкапа:

```bash
docker cp grafana:/var/lib/grafana ./grafana_backup_$(date +%F)
```

Можно автоматизировать через cron:

```bash
0 3 * * * docker cp grafana:/var/lib/grafana /backups/grafana_$(date +%F)
```

---

## 4. Мониторинг нескольких серверов

Чтобы мониторить другой сервер:

1. На втором сервере запустить **Node Exporter**.
2. В `prometheus.yml` добавить:

```yaml
- job_name: 'node_exporter_remote'
  static_configs:
    - targets: ['IP_СЕРВЕРА:9100']
```

3. Перезапустить Prometheus:

```bash
docker restart prometheus
```

---

## 5. Alertmanager — маршрутизация алертов

Можно разделять алерты по важности:

```yaml
route:
  group_by: ['alertname']
  routes:
    - match:
        severity: critical
      receiver: telegram_critical
    - match:
        severity: warning
      receiver: telegram_warning
```

---

## 6. Мини-практика

1. Поменяй пароль Grafana и включи HTTPS.
2. Установи период хранения метрик в Prometheus 15 дней.
3. Настрой резервное копирование дашбордов.
4. Подключи хотя бы ещё один сервер в мониторинг.
5. Сделай два разных канала алертов: `critical` и `warning`.

---

📍 В **Модуле 11** я могу:

* Научить строить SLA/SLI/SLO отчёты в Grafana.
* Настроить автоматический отчёт в Telegram или email.
* Визуализировать доступность сервисов.

---

Хочешь, чтобы я сделал **Модуль 11 про SLA/SLI/SLO и отчёты в Grafana**?


Отлично, идём в сторону SLA и отчётности 📊📨

---

# 📘 Модуль 11 — SLA, SLI, SLO и автоматические отчёты в Grafana

## 🎯 Цель модуля

* Понять разницу между SLA, SLI и SLO.
* Научиться строить панели с доступностью сервисов.
* Сделать автоматическую отправку отчётов в Telegram/email.
* Получить готовые метрики для управления качеством сервиса.

---

## 1. Теория: SLA, SLO, SLI

🔹 **SLA (Service Level Agreement)** — договорённый уровень качества сервиса (например, доступность 99,9% в месяц).
🔹 **SLO (Service Level Objective)** — внутренняя цель, которую команда ставит, чтобы выполнить SLA (например, 99,95%).
🔹 **SLI (Service Level Indicator)** — фактический показатель (например, измеренная доступность за период = 99,97%).

💡 Пример:

* SLA: 99,9% аптайма.
* SLO: 99,95%.
* SLI: реальные метрики аптайма из мониторинга.

---

## 2. Метрики доступности

Prometheus запрос для расчёта доступности HTTP сервиса:

```promql
sum(rate(http_requests_total{code!~"5.."}[30d])) 
/
sum(rate(http_requests_total[30d])) * 100
```

💡 Это покажет % успешных запросов за 30 дней.

---

## 3. Панель в Grafana для SLA/SLO

1. Добавляем **Prometheus** как datasource.
2. Создаём новый дашборд → Panel.
3. Вставляем PromQL (пример выше).
4. Включаем отображение в **Gauge** (шкала).
5. Настраиваем цвет:

   * Зелёный: >99.95
   * Жёлтый: 99.9–99.95
   * Красный: <99.9

---

## 4. Автоматическая отправка отчётов

### Через Telegram Bot

1. Создаём бота через @BotFather.
2. Получаем `BOT_TOKEN` и `CHAT_ID`.
3. В Alertmanager можно отправлять «ежемесячный отчёт» с использованием [Grafana Image Renderer](https://grafana.com/docs/grafana/latest/image-rendering/).

Пример скрипта `send_report.sh`:

```bash
#!/bin/bash
IMG_URL="http://grafana:3000/render/d/abcd1234/my-sla-dashboard?from=now-30d&to=now"
curl -o sla.png "$IMG_URL" --user superadmin:пароль
curl -F chat_id=ВАШ_CHAT_ID -F photo=@sla.png "https://api.telegram.org/botВАШ_BOT_TOKEN/sendPhoto"
```

---

### Через email (Grafana Reporting)

В Grafana → **Settings** → **Reporting**:

* Настраиваем SMTP (например, Gmail SMTP).
* Создаём отчёт → планируем его отправку ежемесячно.

---

## 5. Пример SLO-дашборда

* **Доступность сервиса (%)** — gauge.
* **Количество инцидентов за месяц** — bar chart.
* **Среднее время восстановления (MTTR)** — stat panel.
* **Время безотказной работы (MTBF)** — stat panel.
* **Тренд доступности за 6 месяцев** — line chart.

---

## 6. Мини-практика

1. Построй панель с доступностью сервиса.
2. Настрой шкалу с цветами для SLA/SLO.
3. Сделай ежемесячный отчёт в Telegram с рендером панели.
4. Добавь email-отчёт для менеджеров.

---

📍 В **Модуле 12** я могу:

* Научить строить мониторинг бизнес-метрик (доход, заказы, конверсии).
* Интегрировать Grafana с Google Analytics, PostgreSQL, Kafka.
* Делать алерты не только по технике, но и по бизнесу.

---

Хочешь, чтобы я сделал **Модуль 12 по бизнес-метрикам и интеграциям Grafana**?


Отлично, идём к бизнес-метрикам и интеграциям 💼📊

---

# 📘 Модуль 12 — Мониторинг бизнес-метрик и интеграции Grafana

## 🎯 Цель модуля

* Понять, как мониторить не только сервера, но и бизнес-показатели.
* Научиться подключать внешние источники данных к Grafana.
* Делать алерты по бизнес-метрикам (например, падение числа заказов).
* Строить сквозные дашборды: от инфраструктуры до прибыли.

---

## 1. Зачем бизнес-метрики в Grafana

Технический мониторинг отвечает на вопрос:

> «Сервис работает?»

Бизнес-мониторинг отвечает на вопрос:

> «Сервис приносит результат?»

💡 Иногда сервер живой, но заказы не идут — и это видно только по бизнес-метрикам.

---

## 2. Примеры бизнес-метрик

* **Количество заказов в час**
* **Доход в день**
* **Средний чек**
* **Количество активных пользователей**
* **Конверсия (%)**
* **Время выполнения заказа**

---

## 3. Подключение источников данных

### PostgreSQL (прямой запрос)

1. Добавляем **PostgreSQL** как datasource в Grafana.
2. Настраиваем:

   * Host, Port, User, Password.
   * Database.
3. Пример запроса для панели:

```sql
SELECT
  $__time(date),
  COUNT(*) as orders
FROM orders
WHERE date >= NOW() - INTERVAL '30 days'
GROUP BY date
ORDER BY date;
```

---

### Google Analytics (через плагин)

* Устанавливаем плагин [grafana-google-analytics-datasource](https://grafana.com/grafana/plugins).
* Подключаем GA через API Key.
* Можно строить графики посещаемости, конверсии, источников трафика.

---

### Kafka (метрики событий)

Если бизнес-события идут через Kafka:

* Установить Kafka Exporter → Prometheus → Grafana.
* Можно отслеживать количество транзакций в реальном времени.

---

## 4. Алерты по бизнес-метрикам

Пример: алерт, если заказов меньше 50 в час.

```promql
orders_per_hour < 50
```

или в SQL Panel Grafana:

```sql
SELECT
  COUNT(*) < 50 as low_orders
FROM orders
WHERE date >= NOW() - INTERVAL '1 hour';
```

Дальше в Grafana → **Alert rules** → отправка в Slack/Telegram.

---

## 5. Сквозной бизнес-дашборд

Пример структуры:

1. **KPI в реальном времени** — заказы, выручка, пользователи онлайн.
2. **График конверсии** — % из посетителей, которые сделали заказ.
3. **Воронка продаж** — шаги клиента.
4. **География заказов** — карта.
5. **Технический статус** — uptime, ошибки 5xx, latency.

💡 Это позволяет одновременно видеть, как техника влияет на бизнес.

---

## 6. Мини-практика

1. Подключи PostgreSQL или MySQL к Grafana.
2. Построй график заказов за последние 30 дней.
3. Добавь алерт, если заказов <50 в час.
4. Создай сквозной дашборд с KPI и техническими метриками.

---

📍 В **Модуле 13** я могу:

* Рассказать, как автоматизировать сбор метрик с любых API.
* Сделать мониторинг без базы данных — только через Prometheus + exporters.
* Добавить предиктивную аналитику (прогнозы) в Grafana.

---

Хочешь, чтобы я сделал **Модуль 13 про автоматизацию сбора данных и прогнозирование в Grafana**?
